Exploring all runnable functionalities langchain

In [58]:
from langchain.schema.runnable import(
    RunnableBranch,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough
)
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable.passthrough import RunnableAssign
from functools import partial
from operator import itemgetter

Runnable pass through maps the previous values to the next as it is usually used for branching


In [104]:
# Pass through examples
skip = RunnableLambda(lambda x: x)
skip_alt  = RunnablePassthrough()
# Runnable lambda to convert functions into chainnable components
def add_preface(x, preface = ""):
    if type(x) == str:
        print(f"{preface}{x}")
    else:
        print(x)
    return x
preface_link= RunnableLambda(partial(add_preface, preface="1:"))

def Rpreface(preface=""):
    return RunnableLambda(partial(add_preface, preface=preface))

chain = skip | preface_link | Rpreface("Check: ")
chain.invoke("hello")


1:hello
Check: hello


'hello'

Prompt Templates as  strings initializers

In [53]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
        ("system", "{test} this doesnt works"),
        ("user", "{input}, this works")
    ]
)
chain2 = prompt
chain2.invoke({"test": "system,", "input": "hi"})

ChatPromptValue(messages=[SystemMessage(content='system, this doesnt works'), HumanMessage(content='hi, this works')])

Multi-component chain

In [111]:
def creat_dict(v, key):
    print(type(v), key)
    if isinstance(v, dict):
        return v
    return {key: v}

def RInput(key="input"):
    return RunnableLambda(partial(creat_dict, key=key))

def ROutput(key="output"):
    return RunnableLambda(partial(creat_dict, key=key))

multi_link_chain1 = (
    # passing input as a word or dict, word converts to dict
    RInput()  
    | skip_alt # pass through
    | itemgetter("input") # extracting string, from dict, with key input  
    | Rpreface("step 1: ") # printing with prefix step
    # extracting each word
    | {
        "word1": (lambda x: x.split()[0]),
        "word2": (lambda x: x.split()[1]),
        "word3": (lambda x: x.split()[2]),
        "complete": (lambda x:x)     
    }
    #extracting word by key
    | itemgetter("word1")
    | Rpreface("step 2: ")
    # capitalizing the word
    | (lambda x: x.upper())
    | Rpreface("step 3: ")         
)
multi_link_chain1.invoke({"input": "testing basic multi chain"})
multi_link_chain1.invoke("testing basic multi chain")



<class 'str'> input
step 1: testing basic multi chain
step 2: testing
step 3: TESTING


'TESTING'

In [112]:
multi_link_chain2 = (
    Rpreface("A: ")
    ## Custom ensure-dictionary process
    | RInput()
    | Rpreface("B: ")
    ## Pull-values-from-dictionary utility
    | itemgetter("input")
    | Rpreface("C: ")
    ## Anything-in Dictionary-out implicit map
    | {
        'word1' : (lambda x : x.split()[0]),
        'word2' : (lambda x : x.split()[1]),
        'words' : (lambda x: x),  ## <- == to RunnablePassthrough()
    }
    | Rpreface("D: ")
    | itemgetter("word1")
    | Rpreface("E: ")
    ## Anything-in anything-out lambda application
    | RunnableLambda(lambda x: x.upper())
    | Rpreface("F: ")
    ## Custom ensure-dictionary process
    | ROutput()
)

multi_link_chain2.invoke({"input" : "Hello World"})

{'input': 'Hello World'}
<class 'dict'> input
{'input': 'Hello World'}
C: Hello World
{'word1': 'Hello', 'word2': 'World', 'words': 'Hello World'}
E: Hello
F: HELLO
<class 'str'> output


{'output': 'HELLO'}